In [14]:
import rawpy
import imageio
import numpy as np
from PIL import Image
from spectral import *
import spectral.io.aviris as aviris
import matplotlib
matplotlib.use('WXAgg')
import matplotlib.pyplot as plt
import wx
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from scipy.signal import argrelextrema
import scipy
import cv2
from sklearn import svm
import pickle
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import random
import cv2
spectral.settings.WX_GL_DEPTH_SIZE = 16


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution1D, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from keras.initializers import random_uniform

In [ ]:
import tensorflow as tf


tf.config.run_functions_eagerly(True)

Preprocessing

In [16]:
w = 2048
band = 98

lambda_list = [
 601.00,
 602.00,
 606.00,
 610.00,
 612.36,
 616.73,
 621.20,
 624.07,
 626.71,
 628.05,
 631.63,
 636.59,
 641.24,
 646.58,
 651.24,
 654.85,
 658.63,
 660.78,
 665.41,
 669.57,
 673.58,
 683.94,
 688.45,
 692.56,
 696.90,
 699.05,
 703.90,
 708.03,
 712.41,
 719.38,
 724.03,
 727.76,
 731.65,
 734.10,
 737.94,
 741.27,
 744.62,
 752.29,
 758.12,
 762.98,
 767.84,
 770.30,
 775.42,
 780.03,
 784.55,
 790.34,
 795.45,
 799.87,
 804.17,
 806.41,
 810.84,
 814.31,
 817.33,
 827.88,
 832.47,
 836.32,
 840.31,
 842.50,
 846.97,
 850.85,
 854.61,
 859.38,
 863.32,
 866.84,
 870.24,
 872.16,
 875.86,
 879.01,
 881.74,
 886.01,
 889.95,
 893.54,
 897.35,
 899.58,
 903.55,
 906.80,
 909.86,
 914.24,
 917.38,
 920.24,
 923.17,
 924.90,
 928.03,
 930.66,
 933.14,
 942.38,
 945.32,
 947.88,
 950.70,
 952.39,
 955.99,
 959.30,
 962.35,
 966.54,
 969.13,
 971.34,
 973.58,
 975.08,
]


dic_wave_length = {}
for k in range(band):
    dic_wave_length[k] = lambda_list[k] 

# for key, value in dic_wave_length.items():
#     if value<701 and value >699:
#         print((key, value))

In [17]:
def create_raw_cube(path, w=w, band=band):
    data_before_process = np.fromfile(path, dtype=np.uint16)
    # data_before_process = np.fromfile(path)
    h = int(len(data_before_process)/(w*band))

    data_1 = np.reshape(data_before_process, (h, w*band))
    
    cube = np.zeros((h, w, band), dtype = np.uint16)
    for k in range(h):
        cube[k,:,:] = np.transpose(np.reshape(data_1[k,:], (band, w)))
    return cube

def background_removal(cube_of_interest):
    h = int((np.shape(cube_of_interest)[0]))
    cropped_cube = np.zeros((h,w,band))
    vi = ndvi(cube_of_interest, 25, 49)
    vi_mask = np.where(vi < 0.3, 0, 1)
    for k in range(band):
        cropped_cube[:,:,k] = np.multiply(cube_of_interest[:,:,k],vi_mask)
    return cropped_cube

def find_interesting_band_with_successive_var(cropped_cube_of_interest): 
    var_list = []
    for k in range(band):
        var_list.append(np.var(cropped_cube_of_interest[:,:,k]))
    dvar = np.gradient(var_list, 1)
    print(argrelextrema(dvar, np.greater))
    return dvar

def create_smooth_cube(cube_of_interest):
    dim = np.shape(cube_of_interest)
    smoothed_cube = np.zeros((dim[0], dim[1], dim[2]), dtype=np.uint8)
    # for i in range(dim[0]):
    #     for j in range(dim[1]): 
    #         smoothed_cube[i,j,:] = scipy.signal.savgol_filter(cropped_cropped_cube[i,j,:], deriv = 1, polyorder = 3, window_length = 29)
    
    smoothed_cube[:,:,:] = scipy.signal.savgol_filter(cube_of_interest[:,:,:], axis = 2, deriv = 0, polyorder = 3, window_length= 9)
    return smoothed_cube

def create_final_cube (path):
    cube = create_raw_cube(path)
    cropped_cube = background_removal(cube)[:,250:1750,:]
    smoothed_cube = create_smooth_cube(cropped_cube)
    return smoothed_cube

def signaltonoise(a, axis=0, ddof=0):
    vect = np.matrix.flatten(a)
    m = np.mean(vect)
    sd = np.std(vect)
    return np.where(sd == 0, 0, m/sd)

def denoise_image(cube_of_interest):
    dim = np.shape(cube_of_interest)
    print(dim)
    denoised_image = np.zeros((dim[0], dim[1], dim[2]), dtype='uint8')
    for k in range(dim[2]):
        slice = cube_of_interest[:,:,k]
        index = np.nonzero(slice)
        created_slice = np.zeros((dim[0], dim[1]))
        created_slice[index] = denoise_wavelet(slice[index], channel_axis = None,
                            method='BayesShrink', mode='soft',
                            rescale_sigma=True)
        # sliceCopy = np.uint8(slice)
        # for k in range(len(index[0])):
        #     created_slice[index[0][k]][index[1][k]] = cv2.fastNlMeansDenoising(sliceCopy[index])
        denoised_image[:,:,k] = created_slice
    return denoised_image

def create_LLSI_and_CCTR1_images(cube_of_interest):
    LLSI = np.divide(cube_of_interest[:,:,29]-cube_of_interest[:,:,0], cube_of_interest[:,:,29]+cube_of_interest[:,:,0]) - cube_of_interest[:,:,54]
    CTR1 = np.divide(cube_of_interest[:,:,25], cube_of_interest[:,:,0])

    
    arr8 = ((LLSI-np.mean(LLSI))/np.var(LLSI)*256).astype(np.uint8)
    # print(np.shape(LLSI))
    heatmap = cv2.applyColorMap(arr8, cv2.COLORMAP_RAINBOW)
    imshow( heatmap)
    imshow(LLSI)



Cube creation (background removal, crop the edges and spectral smoothing)

In [18]:
cube_YR_1 = create_final_cube('Vuka_YR_31_days_low/image_1.raw')


In [11]:
cube_YR_2 = create_final_cube('Vuka_YR_31_days_low/image_2.raw')

In [100]:

cube_YR_3 = create_final_cube('vuka_yellowrust_multileaves_1_1.raw')


In [19]:
cube_healthy_1 = create_final_cube('Vuka Healthy 31 days/image_1.raw')

Trying to remove the noise

In [ ]:
denoise_cube_YR_1 = denoise_image(cube_YR_1)

In [128]:
snr_list = []
for k in range(band):
    slice = cube_YR_1[:,:,k]
    index = np.nonzero(slice)
    snr_list.append(signaltonoise(slice[index]))
    

In [184]:
snr_list_denoise = []
for k in range(band):
    slice = denoise_cube_YR_1[:,:,k]
    index = np.nonzero(slice)
    snr_list_denoise.append(signaltonoise(slice[index]))
    

Research of bands of interest (with ttest first, and then with pca)

In [23]:
ttest_list = []
for k in range(band):
    slice_1 = cube_healthy_1[:,:,k]
    slice_2 = cube_YR_1[:,:,k]
    index_1 = np.nonzero(slice_1)
    index_2 = np.nonzero(slice_2)
    ttest_list.append(scipy.stats.ttest_ind(slice_1[index_1], slice_2[index_2])[0])



In [270]:
list_max_ttest = argrelextrema(np.array(ttest_list), np.greater)
plt.figure()
plt.plot(lambda_list, ttest_list)

In [24]:
pca = principal_components(cube_YR_3)

In [25]:
list_contribution_pca = []
for i in range(band):
    list_contribution_pca.append(pca.cov[i,i])

In [26]:
adjusted_ttest = (ttest_list + 500*np.ones(98))*10
plt.figure()
plt.plot(lambda_list, list_contribution_pca, label = "variance selection (PCA)")
plt.plot(lambda_list, adjusted_ttest, label = "t test selection")
plt.legend()
# argrelextrema(np.array(list_contribution), np.greater)


In [28]:
plt.figure()
plt.plot(lambda_list, list_contribution_pca, label = "variance selection (PCA)")
plt.legend()

Extract pixels from the image, and give a label

In [81]:
def create_data(cube_of_interest, label_of_interest = 0):
    ## take the non zeros pixels and normalise with features
    idx = np.nonzero(cube_of_interest[:,:,0])
    non_zero_pixels = np.array(cube_of_interest)[idx[0], idx[1], :]
    non_zero_pixels_band_reduced = non_zero_pixels[:,[45,53,66,75,96]]
    ## size = (nb of pixels, 5)
    normalized_cube = preprocessing.normalize((non_zero_pixels_band_reduced), axis =0)
    labels = np.ones((len(idx[0])))*label_of_interest
    return normalized_cube, labels 

In [141]:
def extract_batch(X_all, Y_all, batch_size, train = True):
    if train:
        
        nb_YR = np.count_nonzero(Y_all)
        nb_tot = len(X_all)
        nb_healthy = nb_tot - nb_YR
        random_index = random.sample(range(0,nb_healthy), int(batch_size/2)) + random.sample(range(nb_healthy,nb_tot), int(batch_size/2))
        X = np.zeros((batch_size, 5))
        X[:,:] = X_all[[random_index],:]
        Y_list = np.array(Y_all)[[random_index]]
        Y = np.asarray(Y_list).astype('float32').reshape((-1,1))
        
        return X, Y, random_index
    else: 
        X = np.zeros((batch_size, 5))
        random_index = random.sample(range(0,len(X_all)), int(batch_size))
        X[:,:] = X_all[[random_index],:]
        return X, random_index

Creation of X_train and Y_train

In [128]:
X_train_healthy, Y_train_healthy = create_data(cube_healthy_1, 0)
X_train_YR, Y_train_YR = create_data(cube_YR_1, 1)


X_train_all = np.concatenate((X_train_healthy, X_train_YR))
Y_train_all = np.concatenate((Y_train_healthy, Y_train_YR))

Stock variables in an external file

In [59]:

# variables = [X_train_raw, Y_train_raw, cube_healthy_1, cube_YR_1]
# import pickle
# Savingvariables = open("data.txt","wb")
# pickle.dump(variables, Savingvariables)
# Savingvariables.close()

## if I want to take them back
# fichierini = "data.txt"
# fichierSauvegarde = open(fichierini,"rb")
# variables = pickle.load(fichierSauvegarde)
# X_train_raw = variables[0]
# Y_train_raw = variables[1]

Take a smaller training set (computational purpose)

In [132]:
X_train, Y_train, random_index_train = extract_batch(X_train_all, Y_train_all, 10000)

C:\Users\Manon\AppData\Local\Temp/ipykernel_8852/248312806.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y_list = np.array(Y_all)[[random_index]]


Creation of the testing dataset

In [143]:
X_test_all = create_data(cube_YR_3)[0]

In [142]:
X_test, random_index_test = extract_batch(X_test_all, [], 10000, train = False)

Creation of the model

In [134]:
model = Sequential()
# model.add(Convolution1D(nb_filter=32, filter_length=3, input_shape= (32, len(X_train), 5), activation='relu'))

### 10 000 samples, batch size = 32, 5 wavelengths
model.add(Convolution1D(filters=32, kernel_size =3, input_shape= (5, 1), activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Convolution1D(filters=16, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [136]:
opt = SGD(lr=0.000001)
model.compile(loss='binary_crossentropy', optimizer= opt, metrics=['acc'])

C:\Users\Manon\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Training

In [137]:
model.fit(X_train, Y_train, epochs=5, batch_size=128)

C:\Users\Manon\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
79/79 [==============================] - 3s 30ms/step - loss: 0.6931 - acc: 0.4736
Epoch 2/5
79/79 [==============================] - 2s 26ms/step - loss: 0.6931 - acc: 0.4819
Epoch 3/5
79/79 [==============================] - 2s 21ms/step - loss: 0.6931 - acc: 0.4715
Epoch 4/5
79/79 [==============================] - 2s 26ms/step - loss: 0.6931 - acc: 0.4744
Epoch 5/5
79/79 [==============================] - 3s 33ms/step - loss: 0.6931 - acc: 0.4800: 1s - loss


In [107]:
Y_predicted = model.predict(X_train)

NameError: name 'model' is not defined

Convert probability into class

In [442]:
Y_predicted = 1*(Y_predicted>0.5)


accuracy_score(Y_predicted[:,:,0], Y_train.astype(np.int))

C:\Users\Manon\AppData\Local\Temp/ipykernel_1216/2484113428.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  accuracy_score(Y_predicted[:,:,0], Y_train.astype(np.int))


0.4972

Testing with the CNN

In [144]:
Y_test = model.predict(X_test)

C:\Users\Manon\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [145]:
Y_test_binary = (Y_test > 0.5)

In [156]:
def create_class_map(cube_of_interest, random_index, prediction):
    idx = np.transpose(np.nonzero(cube_of_interest[:,:,0]))
    classe = np.ones(np.shape(cube_of_interest[:,:,0]))*2
    j=0
    for k in random_index:
        classe[idx[k][0],idx[k][1]] = prediction[j]
        j+=1
    return(classe)

In [152]:
view = imshow(cube_YR_3, (20,10,5), classes = classe_test)
view.set_display_mode('overlay')
view.class_alpha = 0.8

Testing with a SVM

In [154]:
clf = svm.SVC()
clf.fit(X_train, Y_train)

C:\Users\Manon\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [155]:
prediction_SVM = clf.predict(X_test)

In [157]:
classe_SVM = create_class_map(cube_YR_3, random_index_test, prediction_SVM)

In [185]:
imshow(cube_YR_3, classes = np.where(classe_SVM ==2, 1, 0))

ImageView object:
  Display bands       :  [0, 49.0, 97]
  Interpolation       :  <default>
  RGB data limits     :
    R: [0.0, 234.0]
    G: [0.0, 255.0]
    B: [0.0, 255.0]

In [178]:
print(np.shape(np.where(classe_SVM == 2)))

(2, 1982000)
